1\. Write a function that converts number representation (bin<->dec<->hex)

In [26]:
def myconvert():
    m=input("Would you like to convert a decimal number, a binary one or an hexagesimal? Type in dec, bin or hex :) \n")
    a=input("Which number would you like to convert? \n")
    if m=="dec":
    #converto da decimale a binario
        a=int(a)
        b=a
        mybin=[]
        while a!=0:
            mybin.append(int(a%2))
            a=int(a/2)
        mybin=mybin[::-1]  
        
     #converto da decimale a hex
        myhex=[]
        while b!=0:
            myhex.append(int(b%16))
            b=int(b/16)
            mydict=dict([(10, 'a'), (11, 'b'), (12, 'c'), (13, 'd'), (14, 'e'), (15, 'f')])
            myhex=[mydict.get(n, n) for n in myhex]
            myhex=myhex[::-1]
        return print("Your number in binary is %s, in hexagesimal is %s"% (mybin, myhex))      

    elif m=="bin":
    #converto da binario a decimale
        a=list(a)
        a=a[::-1] 
        mydec=0
        for i in range(0,len(a)):
            mydec+=int(a[i])*(2**i)
    #per pigrizia converto da decimale a hex col codice di prima
        myhex=[]
        mydec1=mydec
        while mydec1!=0:
            myhex.append(int(mydec1%16))
            mydec1=int(mydec1/16)
            mydict=dict([(10, 'a'), (11, 'b'), (12, 'c'), (13, 'd'), (14, 'e'), (15, 'f')])
            myhex=[mydict.get(n, n) for n in myhex]
            myhex=myhex[::-1]        
        return print("Your number in decimal is %d, in hexagesimal is %s"% (mydec, myhex))  
    
    else:
    #converto da sessagesimale a decimale
        mydict=dict([(10, 'a'), (11, 'b'), (12, 'c'), (13, 'd'), (14, 'e'), (15, 'f')])
        mydictinvert={v: k for k, v in mydict.items()}
        mydec=0
        a=a[::-1]  
        a=[mydictinvert.get(n, n) for n in a]
        for i in range(len(a)):
            mydec=int(a[i])(16*i)+mydec
    #per pigrizia converto da decimale a binario col codice di prima
        mybin=[]
        mydec1=mydec
        while mydec1!=0:
            mybin.append(int(mydec1%2))
            mydec1=int(mydec1/2)
        mybin=mybin[::-1]  
    
        return print("Your number in decimal is %d, in binary is %s"% (mydec, mybin))

        
        
myconvert()

Would you like to convert a decimal number, a binary one or an hexagesimal? Type in dec, bin or hex :) 
bin
Which number would you like to convert? 
111001110111
Your number in decimal is 3703, in hexagesimal is ['e', 7, 7]


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [157]:
def my_flow(N):
    under=1.0
    over=1.0
    for i in range(N):
        under=under/2
        over=over*2
    print("undeflow:",under,"\noverflow:", over,"\n")    
    
print("Using N=1023 we find the value for the overflow:")
my_flow(10**3+23)

print("Using N=1074 we find the value for the underflow:")
my_flow(10**3+74)


print("Using N=1O75:")
my_flow(10**3+75)

Using N=1023 we find the value for the overflow:
undeflow: 1.1125369292536007e-308 
overflow: 8.98846567431158e+307 

Using N=1074 we find the value for the underflow:
undeflow: 5e-324 
overflow: inf 

Using N=1O75:
undeflow: 0.0 
overflow: inf 



4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [112]:
print("Machine precision using np.finfo(np.float64).eps:",np.finfo(np.float64).eps)

one=0
eps= 1.0
while one != 1.0:
    eps = eps/2
    one=1.0 + eps
print('Machine precision using my function:', 2*eps)

Machine precision using np.finfo(np.float64).eps: 2.220446049250313e-16
Machine precision using my function: 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [101]:
import numpy as np
import math
def my_quadratic(a,b,c):
    x1=(-b+math.sqrt(b**2-4*a*c))/(2*a)
    x2=(-b-math.sqrt(b**2-4*a*c))/(2*a)
    return x1,x2
print("Solutions (a): \n",my_quadratic(0.001,1000,0.001))

print("\n The mantissa is represented by a limited amount of bits, and in the solution to the quadratic equation we have to \n face the problem that b is close to np.sqrt(b**2-4*a*c).")
print("In fact, with 10 significan digits, b is represented by %.10f, while np.sqrt(b**2-4*a*c) by %.10f.\n"%(1000, np.sqrt(1000**2-4*0.001*0.001)))
def my_quadratic2(a,b,c):
    x1=(-b+math.sqrt(b**2-4*a*c))*(-b-math.sqrt(b**2-4*a*c))/(2*a*(-b-math.sqrt(b**2-4*a*c)))
    x2=(-b-math.sqrt(b**2-4*a*c))*(-b+math.sqrt(b**2-4*a*c))/(2*a*(-b+math.sqrt(b**2-4*a*c)))
    return x1,x2
print("Solutions (b): \n",my_quadratic2(0.001,1000,0.001))
print("\nThe value of the second root for the two expressions is different. We are diving a big number by a small one. \nBecause of float rounding the associative law does not necessarily hold. \n")

print("Solutions (c): \n")
print("I decided to move to the log scale in order to get a more accurate solution. \nI implemented the log scaling in both equations. \n")
def my_quadratic3(a,b,c):
    
    x1=math.log10(abs((-b+math.sqrt(b**2-4*a*c))))-np.log10(abs((2*a)))
    x1=(10**x1)*np.sign((-b+math.sqrt(b**2-4*a*c))/(2*a))
    x2=math.log10(abs((-b-math.sqrt(b**2-4*a*c))))-math.log10(abs((2*a)))
    x2=(10**x2)*np.sign((-b-math.sqrt(b**2-4*a*c))/(2*a))
    return x1,x2

def my_quadratic4(a,b,c):
    num_sum= -b+math.sqrt(b**2-4*a*c)
    num_diff=-b-math.sqrt(b**2-4*a*c)
    
    x1=math.log10(abs(num_sum))+math.log10(abs(num_diff))-math.log10(abs(num_diff))-math.log10(2*a)
    x1=(10**x1)*np.sign((-b+math.sqrt(b**2-4*a*c))*a)
    
    x2=math.log10(abs(num_diff))+math.log10(abs(num_sum))-math.log10(abs(num_sum))-math.log10(2*a)
    x2=(10**x2)*np.sign(num_diff*a)
  
    return x1,x2
print(my_quadratic3(0.001,1000,0.001), "\n",my_quadratic4(0.001,1000,0.001))

Solutions (a): 
 (-9.999894245993346e-07, -999999.999999)

 The mantissa is represented by a limited amount of bits, and in the solution to the quadratic equation we have to 
 face the problem that b is close to np.sqrt(b**2-4*a*c).
In fact, with 10 significan digits, b is represented by 1000.0000000000, while np.sqrt(b**2-4*a*c) by 999.9999999980.

Solutions (b): 
 (-9.999894245993346e-07, -999999.9999990001)

The value of the second root for the two expressions is different. We are diving a big number by a small one. 
Because of float rounding the associative law does not necessarily hold. 

Solutions (c): 

I decided to move to the log scale in order to get a more accurate solution. 
I implemented the log scaling in both equations. 

(-9.999894245993342e-07, -999999.999999) 
 (-9.999894245993342e-07, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [187]:
def f(x):
    return x*(x-1)

def df(x,delta):
    return (f(x+delta)-f(x))/delta
print("The analytical derivative of f(x) in x=1 for 𝛿=10^−2 is:","1")
print("The computed derivative in x=1 for 𝛿=10^−2 is:",df(1,10**-2))
print("I think that the difference is mainly due to the fact that I am using 𝛿=10^−2, which is quite different from 0.\n")

print("The derivative in x=1 for 𝛿=10^−4 is:",df(1,10**-4)," error:", 1-df(1,10**-4))
print("The derivative in x=1 for 𝛿=10^−6 is:",df(1,10**-6)," error:", 1-df(1,10**-6))
print("The derivative in x=1 for 𝛿=10^−8 is:",df(1,10**-8)," error:", 1-df(1,10**-8))
print("The derivative in x=1 for 𝛿=10^−10 is:",df(1,10**-10)," error:", 1-df(1,10**-10))
print("The derivative in x=1 for 𝛿=10^−12 is:",df(1,10**-12)," error:", 1-df(1,10**-12))
print("The derivative in x=1 for 𝛿=10^−14 is:",df(1,10**-14)," error:", 1-df(1,10**-14), "\n")

print("As we can see, we have the best accuracy for 𝛿=10^−8. That is probably the best compromise between a value of 𝛿 that")
print("is both low enough to be considered null and high enough not to have problems subtracting floats that are too close  in value (f(x+𝛿)-f(x)).")

The analytical derivative of f(x) in x=1 for 𝛿=10^−2 is: 1
The computed derivative in x=1 for 𝛿=10^−2 is: 1.010000000000001
I think that the difference is mainly due to the fact that I am using 𝛿=10^−2, which is quite different from 0.

The derivative in x=1 for 𝛿=10^−4 is: 1.0000999999998899  error: -9.999999988985486e-05
The derivative in x=1 for 𝛿=10^−6 is: 1.0000009999177333  error: -9.99917733279787e-07
The derivative in x=1 for 𝛿=10^−8 is: 1.0000000039225287  error: -3.922528746258536e-09
The derivative in x=1 for 𝛿=10^−10 is: 1.000000082840371  error: -8.284037100736441e-08
The derivative in x=1 for 𝛿=10^−12 is: 1.0000889005833413  error: -8.890058334132256e-05
The derivative in x=1 for 𝛿=10^−14 is: 0.9992007221626509  error: 0.0007992778373491216 

As we can see, we have the best accuracy for 𝛿=10^−8. That is probably the best compromise between a value of 𝛿 that
is both low enough to be considered null and high enough not to have problems subtracting floats that are too close 

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [233]:
import numpy as np
def my_riem(N):
    I=0
    for k in range(N):
        I=np.sqrt(1-(k/N)**2)+I
    return (I*2/N)

print("For 𝑁=100 I get:", my_riem(10**2),", which has a %error:", ((my_riem(10**2)-np.pi/2)/np.pi/2),"%")
%timeit my_riem(10**2)
print("For 𝑁=4*10^7, I run it for a minute and get:", my_riem(4*10**7),", which has a %error:", ((my_riem(4*10**7)-np.pi/2)/np.pi/2),"%")
%timeit my_riem(4*10**7)

For 𝑁=100 I get: 1.5802085158895225 , which has a %error: 0.0014979964197253484 %
151 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
For 𝑁=4*10^7, I run it for a minute and get: 1.5707963517926005 , which has a %error: 3.9785081427224914e-09 %
1min ± 686 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
